In [1]:
# stock.py - Stock class to hold data about the stock 
#                    and its price data
import pandas as pd

class Stock(object):
    def __init__(self, symbol, data):
        self.symbol = symbol
        self.data = data
        self.data.Date = pd.to_datetime(self.data.Date, format="%Y-%m-%d")
        self.compute_stats()

#     def compute_stats(self, price_header='Close'):
#         # daily precentage change
#         self.open_diffs = self.data[price_header].pct_change()[1:]

#         # average daily change (%)
#         self.avg = self.open_diffs.mean()
#         # variance of daily percentage changes
#         self.var = self.open_diffs.var()
#         # standard deviation of daily percentage changes
#         self.std = self.open_diffs.std()
        
    def compute_stats(self, price_header='Close'):
        self.open_diffs = self.data[price_header].pct_change()[1:]

        self.avg = self.open_diffs.mean()
        self.var = self.open_diffs.var()
        self.std = self.open_diffs.std()

        # .94 recommended by RiskMetrics (https://www.investopedia.com/articles/07/ewma.asp)
        self.emvar = self.open_diffs.ewm(alpha=0.94).var().iloc[-1]
        self.ema = self.open_diffs.ewm(alpha=0.94).mean().iloc[-1]
        self.emstd = self.open_diffs.ewm(alpha=0.94).std().iloc[-1]

    def get_sym(self):
        return self.symbol

    def get_last_price(self, price_header='Close'):
        return self.data[price_header].iloc[-1]

    def get_daily_return_avg(self):
        return self.avg

    def get_daily_return_variance(self):
        return self.var

    def get_daily_return_std(self):
        return self.std

In [2]:
def run(self, ticker_list, simulations, final_day, data_months=1, plot_results=True):
        # number of paths
        self.simulations = simulations
        # last day of simulation 
        self.final_day = datetime.datetime.strptime(final_day, "%Y-%m-%d")
        # number of months worth of data to use
        self.data_months = data_months     
        self.current = datetime.datetime.today()

        self.T = (self.final_day - self.current).days + 2
        if self.T <= 0:
            print("Final day is on or after the current day.")
            exit()

        # weekends don't exist
        self.removed_dates = []
        for d in range(self.T):
            date = (self.current + relativedelta(days=d))
            if date.weekday() > 4:
                self.removed_dates.append(date)
                self.T -= 1

        dh = YahooFinanceAPI(interval=Interval.DAILY)
        symbols = ticker_list
        portfolio = {}

        # get data for all of the tickers
        past = self.current - relativedelta(days=int(self.data_months*30))
        portfolio = dh.get_data_for_tickers(symbols, past, self.current)
        for sym in symbols:
            # create Stock objects for each ticker
            portfolio[sym] = Stock(sym, portfolio[sym])

        paths = {}
        for sym in symbols:
            stock = portfolio[sym]
            s_0 = stock.get_last_price()
            sig = stock.get_daily_return_std()
            sig2 = sig**2
            mu = stock.get_daily_return_avg()
            
        paths = {}
        for sym in symbols:
            stock = portfolio[sym]
            s_0 = stock.get_last_price()
            sig = stock.emstd
            sig2 = stock.emvar
            mu = stock.ema

            # random daily shocks to the stock price (price fluctuations)
            shocks = {sim: random.normal(0, 1, self.T) for sim in range(self.simulations)}
            # cumulative sum of all previous shocks to the stock price (price path)
            brownian_motion = {sim: shocks[sim].cumsum() for sim in range(self.simulations)}
            paths[stock.get_sym()] = [[s_0] for _ in range(self.simulations)]

            # create a simulated price path using the stock price's
            # statistics and the random shocks from above
            for i in range(self.simulations):
                paths[stock.get_sym()][i].extend(\
                    [s_0 * np.exp(((mu - 0.5*sig2)*j) + sig*brownian_motion[i][j-1]) for j in range(1, self.T+1)]
                )     
                               
        
        # defined below in Full Code section
        if plot_results:
            self.plot(paths, portfolio, symbols)
        return paths

In [3]:
if __name__ == '__main__':
    gbm_mc = GBMMonteCarlo()
    gbm_mc.run(["mlr", "gntx"], 50, '2020-10-29', data_months=2)

NameError: name 'GBMMonteCarlo' is not defined

In [ ]:
if __name__ == '__main__':
    sims = 500
    mc_sim = GBMMonteCarlo()
    ticker_list = ["ge", "intc", "gntx", "INO"]
    # get results for each ticker
    results = mc_sim.run(ticker_list, sims, '2020-10-23', data_months=0.5, plot_results=False)

    # high and low strikes for each ticker
    high_strike_price_dict = {"ge": 6.5, "intc": 52.0, "gntx": 26.0, "INO": 10.50}
    low_strike_price_dict = {"ge": 6.0, "intc": 48.0, "gntx": 22.0, "INO": 10.0}
    for sym in ticker_list:
       # compute the likelihood of ending above, below, or under
       # high and low strikes.
        above_high, below_high = option_probability(sims, results[sym], high_strike_price_dict[sym])
        above_low, below_low = option_probability(sims, results[sym], low_strike_price_dict[sym])
        between = above_low - above_high

        # display the results
        prt_str = "{} Low Price: {}  High Price: {} \
            \n\tBelow Low: {}  Above Low: {}  Between: {}  Below High: {}  Above High: {} \
            \n\tBelow Low: {:.2f}%  Above Low: {:.2f}%  Between: {:.2f}%  Below High: {:.2f}%  Above High: {:.2f}%\n\n"
        print(prt_str.format(sym.upper(), low_strike_price_dict[sym], high_strike_price_dict[sym],
            below_low, above_low, between, below_high, above_high, (below_low/sims)*100,
            (above_low/sims)*100, (between/sims)*100, (below_high/sims)*100, (above_high/sims)*100))

In [ ]:
def option_probability(simulations, results, strike_price):
    """
        Get the number of simulations above and below
        a particular strike price.
    """
    below_count = 0
    above_count = 0
    last_idx = len(results[0]) - 1
    for i in range(simulations):
        price = results[i][last_idx]
        if price < strike_price:
            below_count += 1
        else:
            above_count += 1

    return above_count, below_count